<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Lesson17_Boostings/Optuna_screencast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Общий алгоритм работы с Optuna

In [ ]:
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.2 MB/s eta 0:00:00


1. Определяем целевую функцию objective, через аргументы она будет получать специальный объект trial. С его помощью можно назначать различные гипермараметры, Например, как в примере ниже, мы задаем x в интервале [-10,10].

2. Далее создаем объект обучения с помощью метода optuna.create_study.

3. Запускаем оптимизацию целевой функции objective на 10 итераций n_trials=10. Происходит 10 вызовов нашей функции с различными параметрами от -10 до 10. Какие именно параметры выбирает optuna будет описано ниже.

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=40)

study.best_params

[I 2025-01-22 17:40:47,335] A new study created in memory with name: no-name-01bef281-ee1e-4094-be48-c41669c8282c
[I 2025-01-22 17:40:47,344] Trial 0 finished with value: 100.40715349302097 and parameters: {'x': -8.020336994982802}. Best is trial 0 with value: 100.40715349302097.
[I 2025-01-22 17:40:47,346] Trial 1 finished with value: 131.2387530576059 and parameters: {'x': -9.455948370065478}. Best is trial 0 with value: 100.40715349302097.
[I 2025-01-22 17:40:47,350] Trial 2 finished with value: 41.011747746880324 and parameters: {'x': 8.404041516642465}. Best is trial 2 with value: 41.011747746880324.
[I 2025-01-22 17:40:47,360] Trial 3 finished with value: 112.85515380067778 and parameters: {'x': -8.623330635948303}. Best is trial 2 with value: 41.011747746880324.
[I 2025-01-22 17:40:47,366] Trial 4 finished with value: 65.68673511809047 and parameters: {'x': -6.104735351514599}. Best is trial 2 with value: 41.011747746880324.
[I 2025-01-22 17:40:47,368] Trial 5 finished with valu

{'x': 2.0392553708959795}

## Загрузка данных и импорт библиотек

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score

from sklearn.datasets import fetch_california_housing

In [ ]:
RANDOM_STATE = 42

In [ ]:
!pip install lightgbm -q

In [ ]:
from lightgbm import LGBMRegressor

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
data = fetch_california_housing(as_frame=True)

X = data.data
y = data.target

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

## Подбор гиперпараметров с Optuna

Разобъем данные на тренировочную и тестовую часть. На тренировочной части по кросс-валидации подберем гиперпараметры моделей, а затем проверим качество на тестовой части.

In [ ]:
def objective_lgbm(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)

    score = cross_val_score(LGBMRegressor(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators),
                            Xtrain, ytrain, cv=3, scoring='r2', n_jobs=-1).mean()
    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective_lgbm, n_trials=100)

[I 2025-01-22 17:41:15,745] A new study created in memory with name: no-name-5f49ff2b-a414-4486-9ada-9f5687265e4a
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
[I 2025-01-22 17:41:26,619] Trial 0 finished with value: 0.014144943114676992 and parameters: {'max_depth': 15, 'learning_rate': 5.6295290241378764e-05, 'n_estimators': 199}. Best is trial 0 with value: 0.014144943114676992.
[I 2025-01-22 17:41:32,186] Trial 1 finished with value: 0.024778902490216748 and parameters: {'max_depth': 4, 'learning_rate': 2.5315080282

In [ ]:
import optuna.visualization as vis


vis.plot_param_importances(study).show()
vis.plot_optimization_history(study).show()
vis.plot_parallel_coordinate(study).show()

In [ ]:
study.best_params

{'max_depth': 13, 'learning_rate': 0.04232599692911454, 'n_estimators': 974}

In [ ]:
model = LGBMRegressor(**study.best_params)
model.fit(Xtrain, ytrain)

pred = model.predict(Xtest)

r2_score(ytest, pred)

0.855095804413975